In [11]:
import os
import numpy as np
import rasterio
from rasterio.enums import Resampling
from rasterio.warp import (
    calculate_default_transform,
    reproject,
    Resampling as WarpResampling,
)
import matplotlib.pyplot as plt
import contextily as ctx
from ipywidgets import interact, FloatSlider, IntSlider, Layout

In [12]:
# 1. Paths to rasters (ML map + distance layers + binary mask)

ML_path = r"C:\Users\TyHow\MinersAI Dropbox\Tyler Howe\Sibelco_Stuff\linear_combination_layers\averaged_probability_map_smoothed_thresholded_95th_percentile.tif"
roads_path = r"C:\Users\TyHow\MinersAI Dropbox\Tyler Howe\Sibelco_Stuff\linear_combination_layers\infrastructure\roads_vector.tif"
power_path = r"C:\Users\TyHow\MinersAI Dropbox\Tyler Howe\Sibelco_Stuff\linear_combination_layers\infrastructure\powerlines_vector.tif"
rail_path = r"C:\Users\TyHow\MinersAI Dropbox\Tyler Howe\Sibelco_Stuff\linear_combination_layers\infrastructure\railways_vector.tif"
developed_mask_path = r"C:\Users\TyHow\MinersAI Dropbox\Tyler Howe\Sibelco_Stuff\linear_combination_layers\infrastructure\town_mask_continuous.tif"
protected_areas_path = r"C:\Users\TyHow\MinersAI Dropbox\Tyler Howe\Sibelco_Stuff\linear_combination_layers\infrastructure\protected_areas_vector.tif"

paths = [
    ML_path,
    roads_path,
    power_path,
    rail_path,
    developed_mask_path,
    protected_areas_path,
]

labels = {
    ML_path: "Smoothed ML probability map (95th percentile)",
    roads_path: "Max distance to nearest road (m)",
    power_path: "Max distance to nearest power line (m)",
    rail_path: "Max distance to nearest railway (m)",
    developed_mask_path: "Max number of buildings allowed in each developed cluster",
    protected_areas_path: "Min distance to nearest protected area (m)",
}

In [13]:
# 1) Open reference for grid & metadata
with rasterio.open(paths[0]) as ref:
    ref_meta = ref.meta.copy()
    H, W = ref.height, ref.width
    src_crs = ref.crs
    src_tf = ref.transform

# 2) Read & resample all arrays to that grid
arrays = []
for p in paths:
    with rasterio.open(p) as src:
        resamp = Resampling.nearest if p == developed_mask_path else Resampling.bilinear
        arr = src.read(1, out_shape=(H, W), resampling=resamp).astype(float)
    arrays.append(arr)

# 3) Native-CRS extent
xmin = ref_meta["transform"][2]
ymax = ref_meta["transform"][5]
xmax = xmin + ref_meta["transform"][0] * W
ymin = ymax + ref_meta["transform"][4] * H

# 4) Compute Web-Mercator transform & shape
dst_crs = "EPSG:3857"
transform_3857, w_3857, h_3857 = calculate_default_transform(
    src_crs, dst_crs, W, H, left=xmin, bottom=ymin, right=xmax, top=ymax
)

In [ ]:
# 5) Reprojection helper
def reproject_arr(arr, method):
    dst = np.zeros((h_3857, w_3857), dtype=arr.dtype)
    reproject(
        source=arr,
        destination=dst,
        src_transform=src_tf,
        src_crs=src_crs,
        dst_transform=transform_3857,
        dst_crs=dst_crs,
        resampling=method,
    )
    return dst


# 6) Reproject ML map + infra maps
ml_map = reproject_arr(arrays[0], WarpResampling.bilinear)
infra_maps = [
    reproject_arr(
        m,
        WarpResampling.nearest if p == developed_mask_path else WarpResampling.bilinear,
    )
    for m, p in zip(arrays[1:], paths[1:])
]

# 7) Extent in Web-Mercator (xmin, xmax, ymin, ymax)
x_min = transform_3857.c
y_max = transform_3857.f
x_max = x_min + transform_3857.a * w_3857
y_min = y_max + transform_3857.e * h_3857
extent_3857 = (x_min, x_max, y_min, y_max)

# 8) Prepare slider labels & ranges
infra_names = [os.path.splitext(os.path.basename(p))[0] for p in paths[1:]]
infra_mins = [float(np.nanmin(m)) for m in infra_maps]
infra_maxs = [float(np.nanmax(m)) for m in infra_maps]

name_to_label = {
    os.path.splitext(os.path.basename(p))[0]: lbl for p, lbl in labels.items()
}

max_label = max(name_to_label.values(), key=len)
desc_px = len(max_label) * 7
DESC_W, WIDGET_W = f"{desc_px}px", "800px"

sliders = {}
for name, mn, mx in zip(infra_names, infra_mins, infra_maxs):
    lbl = name_to_label.get(name, name)
    common = dict(
        description=lbl,
        style={"description_width": DESC_W},
        layout=Layout(width=WIDGET_W),
        continuous_update=False,
    )
    if name == os.path.splitext(os.path.basename(developed_mask_path))[0]:
        sliders[name] = IntSlider(
            value=int(mx), min=int(mn), max=int(mx), step=1, **common
        )
    else:
        step = (mx - mn) / 100 or 1
        sliders[name] = FloatSlider(value=mx, min=mn, max=mx, step=step, **common)

protected_name = os.path.splitext(os.path.basename(protected_areas_path))[0]

# 9) Plot function with transparent zeros and basemap
def show_map(**thresholds):
    filtered = ml_map.copy()
    for name, layer in zip(infra_names, infra_maps):
        thr = thresholds[name]
        if name == protected_name:
            # zero out where too close to protected area
            filtered[layer < thr] = 0
        else:
            # zero out where too far from infra
            filtered[layer > thr] = 0

    # mask zeros → transparent
    masked = np.ma.masked_equal(filtered, 0)

    fig, ax = plt.subplots(figsize=(10, 8))
    ax.set_xlim(extent_3857[0], extent_3857[1])
    ax.set_ylim(extent_3857[2], extent_3857[3])

    # choose basemap provider
    try:
        bm = ctx.providers.CartoDB.Positron
    except AttributeError:
        bm = ctx.providers.Stamen.Terrain

    ctx.add_basemap(ax, source=bm, crs=dst_crs)

    im = ax.imshow(masked, extent=extent_3857, origin="upper", cmap="viridis")
    ax.axis("off")
    ax.set_title("ML output within defined cutoffs")
    plt.colorbar(im, label="Relative Deposit probability", ax=ax)
    plt.tight_layout()
    plt.show()


# 10) Launch interactive sliders
interact(show_map, **sliders)

interactive(children=(FloatSlider(value=92.82425001311873, continuous_update=False, description='Max distance …

<function __main__.show_map(**thresholds)>

In [16]:
### EXPORT THE FILTERED MAP ###

# re-load ML metadata to preserve CRS, transform, etc.
with rasterio.open(ML_path) as src:
    out_meta = src.meta.copy()
# we’re writing one band of floats
out_meta.update(dtype=rasterio.float32, count=1)

# recompute the filtered map using current slider values
filtered = ml_map.copy()
for name, layer in zip(infra_names, infra_maps):
    thr = sliders[name].value
    if name == protected_name:
        # in show_map you did layer < thr → zero
        filtered[layer < thr] = 0
    else:
        # everywhere else you did layer > thr → zero
        filtered[layer > thr] = 0

# write it out
out_path = r"C:\Users\TyHow\MinersAI Dropbox\Tyler Howe\Sibelco_Stuff\linear_combination_layers\combination.tif"
with rasterio.open(out_path, "w", **out_meta) as dst:
    dst.write(filtered.astype("float32"), 1)

print(f"Saved filtered map to {out_path}")

CPLE_AppDefinedError: Deleting C:\Users\TyHow\MinersAI Dropbox\Tyler Howe\Sibelco_Stuff\linear_combination_layers\combination.tif failed: Permission denied